# ODE Integrators I: Explicit Methods

Ordinary differential equations form the mathematical foundation of
physics.
They describe how systems evolve in time, e.g., Newton's second law of
motion is
\begin{align}
  f = m a = m \frac{d^2x}{dt^2},
\end{align}
which is a second-order ODE relating force to acceleration.
More generally, an ODE relates a function of time to its time
derivatives.

While ODEs capture the essence of dynamics, analytic solutions are
possible only in special cases.
Most real systems involve nonlinear forces, damping, or external
driving terms that make exact solutions impossible.
Numerical integration allows us to approximate trajectories step by
step and study systems far beyond what can be solved by hand.

## Problem Definition and Types of ODEs

Consider two forms of first-order ODEs:

1. Time-dependent forcing only:
   \begin{align}
     \frac{dx}{dt} = f(t).
   \end{align}
   Here, the derivative depends only on time.
   Solutions are obtained by direct integration:
   \begin{align}
     x(t) = x(t_0) + \int_{t_0}^{t} f(t') \, dt'.
   \end{align}

2. State- and time-dependent dynamics:
   \begin{align}
     \frac{dx}{dt} = f(x, t).
   \end{align}
   Now, the derivative depends on the state $x$ itself.
   The function we want to solve for also appears inside the RHS,
   making direct integration impossible.
   We cannot evaluate the integral without already knowing $x(t)$ at
   intermediate points.

The first type reduces to standard numerical quadrature (trapezoidal
rule, Simpson's rule, etc.), which we studied earlier.
The second case, nonlinear dependence on both $x$ and $t$, is the
typical situation in physics.
Examples include planetary orbits, nonlinear oscillators, chaotic
systems, and interacting biological populations.
In such cases:
* Direct integration is often not feasible, the problem must be solved
  as an initial value problem (IVP).
* Analytic solutions are often unknown or intractable.
* Numerical methods approximate the solution incrementally, using
  small time steps to trace the system's evolution.

By discretizing time and advancing step-by-step, we can model the
behavior of even the most complex systems.
This is the core idea behind numerical ODE integrators.

## Euler's Method

Euler's method is the simplest techniques for numerically solving
ordinary differential equations.
This method provides an easy way to approximate the solution of an IVP
by advancing one small step at a time.

We can apply Euler's method to an ODE of the form:
\begin{align}
  \frac{dx}{dt} = f(x, t), \quad x(t_0) = x_0
\end{align}
where $x_0$ is the initial value of $x$ at time $t = t_0$.
However, as we will see below, it is usually not recommanded in
pratical calculations because of its stability properties.

### Forward (Explicit) Euler Method

There are three simple ways to derive Euler's method.
The easiest way is simply hold $x$ fixed in $f(x, t)$ and apply the
left Reimann sum.
The left Reimann sum is first order in step size by approximating
$f(x, t)$ as a constant.
In this sense, holding $x$ is somewhat "self-consistent" in terms of
approximation order.

We then recall the definition of a deriviative:
\begin{align}
  f(x, t) = \frac{dx}{dt} = \lim_{\Delta t\rightarrow 0}\frac{x(t + \Delta t) - x(t)}{\Delta t}.
\end{align}
If we simply remove the limit and keep the "finite difference" part,
then it is trivial to show
\begin{align}
  x(t + \Delta t) &\approx x(t) + f(x(t), t)\Delta t.
\end{align}
Which is nothing but again the forward Euler method.
While very intuitive, the above two derivations do not formally show
the order of the Euler method.

We may also consider a numerical approximation to the solution of an
ODE.
We approximate the solution at time $t_{n+1} = t_n + \Delta t$ by
using the Taylor expansion:
\begin{align}
  x(t_{n+1}) = x(t_n) + f(x(t_n), t_n) \Delta t + \mathcal{O}(\Delta t^2)
\end{align}
Neglecting the higher-order terms in the expansion, we obtain once
again the Forward Euler Method:
\begin{align}
  x_{n+1} = x_n + f(x_n, t_n) \Delta t
\end{align}
It is thus a step-by-step approach that proceeds by evaluating $f(x,
t)$ at each time point and then advancing to the next point.
It is an explicit method in 1st order.

Let's solve the simple differential equation:
\begin{align}
  \frac{dx}{dt} = \lambda x(t)
\end{align}

This equation has solution
\begin{align}
  x(t) = \exp[\lambda(t-t_0)]
\end{align}

If we choose $\lambda = 1$ and $x(0) = 1$, the solutoin reduces to
$x(t) = \exp(t)$.

In [ ]:
# Let's visualize the solution:

import numpy as np
from matplotlib import pyplot as plt

T = np.linspace(0, 2, 201)
X = np.exp(T)

plt.plot(T, X)

In [ ]:
# Let's implement Euler's method, with history

def Euler(f, x, t, dt, n):
    X = [np.array(x)]
    T = [np.array(t)]
    for _ in range(n):
        X.append(X[-1] + dt * f(X[-1]))
        T.append(T[-1] + dt)
    return np.array(X), np.array(T)

In [ ]:
# Let's test Euler's method

f = lambda x: x

X1, T1 = Euler(f, 1, 0, 0.1, 20)

plt.plot(T, X)
plt.plot(T1, X1, 'o-')
plt.xlabel('t')
plt.ylabel('x')

In [ ]:
# As always, we can study the convergence of the numerical method

def Err1(N=200):
    X1, T1 = Euler(f, 1, 0, 2/N, N)
    X = np.exp(T1)
    return np.max(abs(X1 - X))

N  = np.array([64, 128, 256, 512, 1024])
E1 = np.array([Err1(n) for n in N])

plt.loglog(N, 16/N, label='1/N')
plt.loglog(N, E1, 'o:', label='Forward Euler')
plt.xlabel('N')
plt.ylabel(r'$\text{err} = \max|x_\text{numeric} - x|$')
plt.legend()

### System of ODEs

In computational astrophysics, we often encounter systems governed by
Newton's laws:
\begin{align}
  m \frac{d^2 x}{dt^2} = f(x, t)
\end{align}

This equation is a second-order ordinary differential equation because
it involves the second derivative of $x$ with respect to $t$.
However, it is often more practical to convert second-order ODEs into
a system of first-order ODEs.
To do this, we introduce a new variable, $v = dx/dt$, representing the
velocity of the object.
This substitution allows us to rewrite the second-order ODE as two
coupled first-order ODEs:
\begin{align}
  \frac{dx}{dt} &= v \\
  \frac{dv}{dt} &= \frac{1}{m}f(x, t)
\end{align}

This formulation provides a convenient way for numerical methods,
which are generally well-suited to solving systems of first-order
ODEs.
To further simplify, we can express these equations in vector notation
by defining $\mathbf{x} = [x, v]^t$ and $\mathbf{f} = [v, f/m]^t$.
The system then becomes:
\begin{align}
  \frac{d\mathbf{x}}{dt} = \mathbf{f}(\mathbf{x}, t).
\end{align}
This vector form emphasizes the structure of the system and enables us
to apply general numerical techniques to solve both equations
simultaneously.

To illustrate this approach, let's consider a classic example: the
simple pendulum under gravity.
The motion of a pendulum of length $l$, swinging under gravity $g$,
can be described by the second-order equation:
\begin{align}
  \frac{d^2\theta}{dt^2} + \frac{g}{l} \sin\theta = 0
\end{align}

Here, $\theta(t)$ is the angle of the pendulum with the vertical, and
the term $\sin \theta$ introduces nonlinearity, which makes the
equation challenging to solve analytically.
Converting this equation into a system of first-order ODEs allows us
to handle it more effectively with numerical methods.
We define $\Omega = \frac{d\theta}{dt}$, the angular velocity, leading
to the following system:
\begin{align}
  \frac{d\theta(t)}{dt} &= \Omega(t)\\
  \frac{d\Omega(t)}{dt} &= - \frac{g}{l}\sin\theta(t)
\end{align}

In vector notation, we represent the system as:
\begin{align}
  \frac{d\mathbf{x}(t)}{dt} = \mathbf{f}(\mathbf{x}, t)
\end{align}
where
\begin{align}
  \mathbf{x} &= \begin{bmatrix} \theta(t) \\ \Omega(t) \end{bmatrix}, \\
  \mathbf{f}(\mathbf{x}, t) &= \begin{bmatrix} \Omega(t) \\ -\frac{g}{l} \sin \theta(t) \end{bmatrix}.
\end{align}

In later part of the lecture, we will try to solve this full problem.
But to derive and compare different numerical methods, let's first
reduce the problem to something that has analytical solutions.
Specifically, we can simplify the pendulum problem further by assuming
small oscillations, where the angle $\theta$ is small enough that
$\sin \theta \approx \theta$.
This approximation linearizes the equation of motion, reducing the
system to a simple harmonic oscillator.

In this approximation, the equation of motion becomes:
\begin{align}
  \frac{d^2 \theta}{dt^2} + \frac{g}{l} \theta = 0
\end{align}
As a result, the system of ODEs becomes:
\begin{align}
  \mathbf{x} &= \begin{bmatrix} \theta(t) \\ \Omega(t) \end{bmatrix}, \\
  \mathbf{f}(\mathbf{x}, t) &= \begin{bmatrix} \Omega(t) \\ -\frac{g}{l} \theta(t) \end{bmatrix}.
\end{align}

In [ ]:
# Let's first plot the analytical solution

T     = np.linspace(0, 10, 101)
Theta = 0.01 * np.sin(T)

plt.plot(T, Theta)

In [ ]:
# Thanks to operator overriding, our forward Euler method is almost
# ready to solve system of ODEs!
# There is no need to rewrite it!

In [ ]:
# Compare the analytical and numerical solutions

def F(x):
    theta, omega = x
    return np.array([omega, -theta])

X1, T1 = Euler(F, [0.0, 0.01], 0, 0.01, 1000)

Theta1 = X1[:,0]
Omega1 = X1[:,1]

plt.plot(T,  Theta)
plt.plot(T1, Theta1)

In [ ]:
# Again, we can study the convergence of the numerical method

def Err(integrator, N=100, **kwargs):
    Xf, Tf = integrator(F, [0, 0.01], 0, 10/N, N, **kwargs)
    Thetaf = Xf[:,0]
    Theta  = 0.01 * np.sin(Tf)
    return np.max(abs(Thetaf - Theta))

N  = np.array([64, 128, 256, 512, 1024])
E1 = np.array([Err(Euler, n) for n in N])

plt.loglog(N, 0.5/N, label='1/N')
plt.loglog(N, E1, 'o:', label='Forward Euler')
plt.xlabel('N')
plt.ylabel(r'$\text{err} = \max|x_\text{numeric} - x|$')
plt.legend()

### A Midpoint Method?

The Forward Euler method is only first-order accurate, meaning its
error decreases linearly with the step size $\Delta t$, as we saw in
the convergence plots.
While it is simple to implement, this method's convergence rate is
too slow.
Achieving higher accuracy requires very small steps, which can become
computationally expensive.

This naturally raises the question: can we improve the convergence
rate of our numerical solution, reducing the error more rapidly as we
use smaller steps?

To explore this, we can draw inspiration from our previous work on
numerical integration.
In that context, we observed that the midpoint (or central) Riemann
sum converges faster than the left or right Riemann sums.
This suggests that a midpoint approach may also provide advantages in
solving ODEs.

One possible improvement is to propose a midpoint method that attempts
to evaluate the function $\mathbf{f}$ at the midpoint between steps.
Mathematically, this approach can be expressed as:
\begin{align}
  \mathbf{x}_{n+1} = \mathbf{x}_n + \mathbf{f}(\mathbf{x}_{n+1/2}, t_{n+1/2}) \Delta t
\end{align}

However, a significant problem arises: the midpoint value $f_{n+1/2}
\equiv f(\mathbf{x}_{n+1/2}, t_{n+1/2})$ is unknown at step $n$.
We need to know the value of $\mathbf{x}$ at the midpoint $t_{n+1/2}$
to use this method, but this value cannot be calculated without
already knowing the future values of $\mathbf{x}$.
This issue makes a straightforward midpoint method impractical for
generic ODEs, where $\mathbf{f}$ depends on both $\mathbf{x}$ and $t$.

An exception occurs if $\mathbf{f}$ depends only on $t$, as in
$\frac{dx}{dt} = f(t)$; in such cases, a true midpoint method is
feasible, which is nothing but out middle Reimann sum.
However, for most ODEs, including those where $\mathbf{f}$ depends on
$\mathbf{f}$, a different approach is necessary.

### A Simple Idea: The Second-Order Runge-Kutta Method

To work around this issue, we can approximate the midpoint value
instead of calculating it exactly.
It suffices to find an approximate solution for $\mathbf{x}$ at the
half-step, which we denote as $\mathbf{x}_{n+1/2}$.

The simplest way to do this is to use the Forward Euler method to
compute an estimated value at the midpoint.
Specifically, we can approximate $\mathbf{x}$ at $t_{n+1/2}$ as:
\begin{align}
  \tilde{\mathbf{x}}_{n+1/2} = \mathbf{x}_n + \mathbf{f}(\mathbf{x}_n, t_n) \frac{\Delta t}{2}
\end{align}
Using this half-step approximation, we then proceed with a full step
to find $\mathbf{x}_{n+1}$ by evaluating $f$ at the midpoint:
\begin{align}
  \mathbf{x}_{n+1} = \mathbf{x}_n + f(\tilde{\mathbf{x}}_{n+1/2}, t_{n+1/2}) \Delta t
\end{align}
This approach, known as the second-order Runge-Kutta method,
incorporates midpoint information and achieves second-order accuracy.

This second-order Runge-Kutta method improves convergence by
leveraging an approximate midpoint, resulting in a more accurate
solution than the first-order Euler method without prohibitively small
step size.

In [ ]:
def RK2(f, x, t, dt, n):
    X = [np.array(x)]
    T = [np.array(t)]
    for _ in range(n):
        k1 = f(X[-1])
        xh = X[-1] + 0.5 * dt * k1
        
        k2 = f(xh)
        xf = X[-1] + dt * k2
        
        X.append(xf)
        T.append(T[-1] + dt)
    return np.array(X), np.array(T)

In [ ]:
N=64
X2, T2 = RK2(F, [0, 0.01], 0, 10/N, N)
plt.plot(T, 0.01 * np.sin(T))
plt.plot(T2, X2[:,0], 'o')

In [ ]:
def Euler(f, x, t, dt, n, substep=1):
    if substep == 1:  # the above forward Euler
        X = [np.array(x)]
        T = [np.array(t)]
        for _ in range(n):
            X.append(X[-1] + dt * f(X[-1]))
            T.append(T[-1] + dt)
        return np.array(X), np.array(T)
    else:
        X, T = Euler(f, x, t, dt/substep, n*substep)
        return X[::substep], T[::substep]

In [ ]:
N=64
X1, T1 = Euler(F, [0, 0.01], 0, 10/N, N, substep=2)
plt.plot(T, 0.01 * np.sin(T))
plt.plot(T1, X1[:,0], 'o')

In [ ]:
# Again, we can study the convergence of the numerical method

N   = np.array([64, 128, 256, 512, 1024])
E1  = np.array([Err(Euler, n)            for n in N])
E12 = np.array([Err(Euler, n, substep=2) for n in N])
E2  = np.array([Err(RK2,   n)            for n in N])

plt.loglog(N, 0.5/N, label='1/N')
plt.loglog(N, E1,  'o:', label='Forward Euler')
plt.loglog(N, E12, 'o-', label='Substep Euler')
plt.loglog(N, E2,  'o-', label='RK2')

plt.xlabel('N')
plt.ylabel(r'$\text{err} = \max|x_\text{numeric} - x|$')
plt.legend()

The convergence plot clearly demonstrates that RK2 achieves
second-order accuracy.
This means that the error decreases quadratically with the step size
$\Delta t$.
For example, using 1024 points for the integration not only doubles
the computational cost compared to using 512 points but also improves
the accuracy of the solution by approximately 250 times.
This significant increase in accuracy suggests that higher-order
methods offer substantial benefits, especially for problems requiring
high precision.
Naturally, this brings us to the question: can we improve the accuracy
even further than what RK2 provides?